In [27]:
import pulp
from pulp import lpSum
import numpy as np
import pandas as pd

pulp.LpSolverDefault.msg = 0

In [40]:
N = 3
PTDF = np.array((0.33, 0.67, 0.33, -0.44, 0.44, 0.56, 0, 0, 0)).reshape(3,3).T
RAM = np.array((1000,1000,1000))
cost_per_MW = np.array((100, 200, 300))
demand = np.array((1500, 1500, 1500))

In [41]:
model = pulp.LpProblem("Constrained_maximisation", pulp.LpMinimize)
supply = [
            pulp.LpVariable("z{}".format(i), lowBound=-5000, upBound=5000, cat='Integer') for i in range(N)
        ]

model += lpSum(((supply[i])*cost_per_MW[i] for i in range(N)))

row_products = np.dot(PTDF, supply)
for i in range(PTDF.shape[0]):
    row_product = pulp.lpSum([PTDF[i][j] * (supply[j]-demand[j]) for j in range(PTDF.shape[1])])
    model += row_product <= RAM[i]

model += lpSum((demand-supply[j]) for j in range(N)) == 0
model.solve()

1

In [47]:
print(f"Status: {pulp.LpStatus[model.status]}")
for j in range(N):
    print(f"net poistion in zone {j}: {supply[j].value()-demand[j]}, supply in zone {j}: {supply[j].value()}")

Status: Optimal
net poistion in zone 0: 1999.0, supply in zone 0: 3499.0
net poistion in zone 1: -772.0, supply in zone 1: 728.0
net poistion in zone 2: -1227.0, supply in zone 2: 273.0


In [50]:
[PTDF[i][j] * (supply[j].value()-demand[j]) for j in range(PTDF.shape[1])]

[659.6700000000001, -432.32000000000005, -0.0]

In [11]:
np.multiply(PTDF, RAM)

array([[ 330., -440.,    0.],
       [ 670.,  440.,    0.],
       [ 330.,  560.,    0.]])

In [ ]:
import pulp
from pulp import lpSum
import numpy as np
import pandas as pd